In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predict-ovarian-cancer/Supplementary data 5.xlsx
/kaggle/input/predict-ovarian-cancer/Supplementary data 2.xlsx
/kaggle/input/predict-ovarian-cancer/Supplementary data 4.xlsx
/kaggle/input/predict-ovarian-cancer/Supplementary data 1.xlsx
/kaggle/input/predict-ovarian-cancer/Supplementary data 3.xlsx


In [2]:
import pandas as pd
import numpy as np

df = pd.read_excel("/kaggle/input/predict-ovarian-cancer/Supplementary data 1.xlsx")

def clean_numeric(col):
    if col.dtype == 'object':
        col = col.str.replace('>', '').str.replace('<', '').str.strip()
        col = pd.to_numeric(col, errors='coerce') 
    return col

df = df.apply(clean_numeric)


print(df.head(10))
print("\nColumns:")
print(df.columns.tolist())
print("\nShape:", df.shape)
print("\nTarget 'TYPE' distribution:")
print(df['TYPE'].value_counts())
print(df.describe())

   SUBJECT_ID    AFP     AG  Age   ALB    ALP   ALT   AST  BASO#  BASO%  ...  \
0           1   3.58  19.36   47  45.4   56.0  11.0  24.0   0.01   0.30  ...   
1           2  34.24  23.98   61  39.9   95.0   9.0  13.0   0.02   0.30  ...   
2           3   1.50  18.40   39  45.4   77.0   9.0  18.0   0.03   0.60  ...   
3           4    NaN  16.60   45  39.2   26.0  16.0  17.0   0.05   0.74  ...   
4           5    NaN  19.97   45  35.0   47.0  21.0  27.0   0.01   0.10  ...   
5           6    NaN  17.39   44  32.9  118.0  51.0  32.0   0.02   0.42  ...   
6           7    NaN  21.87   53   NaN    NaN   NaN   NaN   0.02   0.50  ...   
7           8   3.03  24.05   76  50.4   76.0  16.0  23.0   0.04   0.50  ...   
8           9   1.01  23.63   38  36.3   64.0  62.0  47.0   0.00   0.40  ...   
9          10    NaN  16.86   30  40.8   77.0  16.0  12.0   0.08   1.23  ...   

    NEU    PCT   PDW  PHOS  PLT   RBC   RDW  TBIL    TP     UA  
0  76.2  0.090  13.4  1.46   74  2.64  13.7   5.5  73.

In [3]:
from sklearn.impute import SimpleImputer

df = df.drop('SUBJECT_ID', axis=1)

imputer = SimpleImputer(strategy='mean')
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

print("Missing values handled. New head:")
print(df.head())

Missing values handled. New head:
         AFP     AG   Age   ALB   ALP   ALT   AST  BASO#  BASO%   BUN  ...  \
0   3.580000  19.36  47.0  45.4  56.0  11.0  24.0   0.01   0.30  5.35  ...   
1  34.240000  23.98  61.0  39.9  95.0   9.0  13.0   0.02   0.30  3.21  ...   
2   1.500000  18.40  39.0  45.4  77.0   9.0  18.0   0.03   0.60  3.80  ...   
3  28.899681  16.60  45.0  39.2  26.0  16.0  17.0   0.05   0.74  5.27  ...   
4  28.899681  19.97  45.0  35.0  47.0  21.0  27.0   0.01   0.10  4.89  ...   

    NEU   PCT   PDW  PHOS    PLT   RBC   RDW  TBIL    TP     UA  
0  76.2  0.09  13.4  1.46   74.0  2.64  13.7   5.5  73.9  396.4  
1  76.5  0.30  11.2  1.09  304.0  4.89  12.7   6.8  72.0  119.2  
2  69.7  0.13  15.2  0.97  112.0  4.62  12.0  14.8  77.9  209.2  
3  65.5  0.25  17.4  1.25  339.0  4.01  14.6  10.9  66.1  215.6  
4  59.5  0.28  11.9  0.94  272.0  4.40  13.4   5.3  66.5  206.0  

[5 rows x 50 columns]


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop('TYPE', axis=1)
y = df['TYPE']

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (279, 49)
Test shape: (70, 49)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, preds) * 100, "%")
print("\nClassification Report:")
print(classification_report(y_test, preds, target_names=["Malignant (0)", "Benign (1)"]))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, preds))

Accuracy: 94.28571428571428 %

Classification Report:
               precision    recall  f1-score   support

Malignant (0)       0.97      0.91      0.94        34
   Benign (1)       0.92      0.97      0.95        36

     accuracy                           0.94        70
    macro avg       0.94      0.94      0.94        70
 weighted avg       0.94      0.94      0.94        70


Confusion Matrix:
[[31  3]
 [ 1 35]]


In [8]:
!pip install plotly --quiet
import plotly.express as px
import numpy as np

importances = model.feature_importances_
features = X.columns
indices = np.argsort(importances)[::-1]

fig = px.bar(x=importances[indices[:15]], y=features[indices[:15]],
             orientation='h', 
             title='Top 15 Feature Importances for Ovarian Cancer Classification',
             labels={'x': 'Importance Score', 'y': 'Feature'},
             color=importances[indices[:15]],
             color_continuous_scale='Blues')

fig.update_layout(
    width=800, height=600,
    xaxis_title='Importance Level',
    yaxis_title='Features',
    coloraxis_colorbar={'title': 'Importance'},
    annotations=[dict(x=0.5, y=-0.15, xref='paper', yref='paper', text='Higher importance means the feature contributes more to predictions (e.g., HE4 is a key tumor marker).', showarrow=False)]
)

fig.show()

In [9]:
import plotly.figure_factory as ff

cm = [[31, 3], [1, 35]]
labels = ['Malignant', 'Benign']

fig = ff.create_annotated_heatmap(cm, x=labels, y=labels, colorscale='Blues')

fig.update_layout(
    title='Confusion Matrix for Ovarian Cancer Classification',
    xaxis_title='Predicted Label',
    yaxis_title='True Label',
    width=500, height=400,
    annotations=[dict(x=0.5, y=-0.3, xref='paper', yref='paper', text='Accuracy: 94.29%', showarrow=False, font=dict(size=12, color="black"))]
)

fig.show()

In [10]:
df_plot = X.copy()
df_plot['TYPE'] = y.map({0: 'Malignant', 1: 'Benign'})

fig = px.box(df_plot, x='TYPE', y='HE4',  
             title='Distribution of HE4 by Class (Malignant vs Benign)',
             color='TYPE', color_discrete_map={'Malignant': 'red', 'Benign': 'blue'})

fig.update_layout(
    width=700, height=500,
    xaxis_title='Class',
    yaxis_title='HE4 Level (Normalized)',
    annotations=[dict(x=0.5, y=-0.2, xref='paper', yref='paper', text='Higher HE4 levels are strongly associated with malignant cases, making it a key biomarker for diagnosis.', showarrow=False, font=dict(size=12))]
)

fig.show()

In [12]:
from sklearn.metrics import roc_curve, auc

probs = model.predict_proba(X_test)[:, 1]  
fpr, tpr, _ = roc_curve(y_test, probs)
roc_auc = auc(fpr, tpr)

fig = px.line(x=fpr, y=tpr, title='ROC Curve for Ovarian Cancer Classification',
              labels={'x': 'False Positive Rate', 'y': 'True Positive Rate'})

fig.add_shape(type='line', line=dict(dash='dash'), x0=0, y0=0, x1=1, y1=1)

fig.update_layout(
    width=700, height=500,
    annotations=[dict(x=0.5, y=0.2, text=f'AUC = {roc_auc:.2f}', showarrow=False, font=dict(size=12))]
)

fig.show()